<a href="https://colab.research.google.com/github/Twinkle-gawri/NER-NLP/blob/main/NER_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets # A library from HuggingFace to easily load tons of public datasets (NLP, vision, etc.)
!pip install seqeval # A library for evaluating sequence labeling tasks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential,layers
from tensorflow.keras.layers import Dense,Bidirectional,LSTM,TimeDistributed,SimpleRNN,GRU
import seqeval
from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
dataset=load_dataset('conll2003')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
print(dataset)
print(dataset.keys()) # to check what is present in dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})
dict_keys(['train', 'validation', 'test'])


In [ ]:
training = dataset['train']
validation = dataset['validation']
test = dataset['test']

In [ ]:
training_tokens = training['tokens']
training_tags = training['ner_tags']

validation_tokens = validation['tokens']
validation_tags = validation['ner_tags']

test_tokens = test['tokens']
test_tags = test['ner_tags']

* fit_on_texts() = make the dictionary
* texts_to_sequences(["apple banana"]) = use the dictionary to get [5,2]


texts_to_sequences() = Converts your text into numbers based on the word-index that fit_on_texts() created.

In [ ]:
tokenizer=Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(training_tokens)

In [ ]:
train_sequences=tokenizer.texts_to_sequences(training_tokens)
validation_sequences=tokenizer.texts_to_sequences(validation_tokens)
test_sequences=tokenizer.texts_to_sequences(test_tokens)
print(test_sequences)

[[86, 18, 227, 442, 8273, 160, 4, 214, 6, 2764, 1060, 3], [1, 1], [1, 4, 169, 1137, 11075, 1], [227, 554, 2, 878, 5, 55, 4974, 158, 731, 24, 8, 8273, 968, 160, 79, 1641, 6, 8, 117, 435, 312, 133, 13, 96, 3], [37, 214, 1274, 55, 6116, 11877, 202, 6, 2, 80, 133, 5, 2, 117, 4, 6463, 7, 8, 2764, 789, 1060, 7, 18159, 1, 3], [214, 3024, 279, 5, 2, 133, 9, 1274, 415, 3545, 3180, 406, 2, 7731, 340, 81, 1, 1009, 1, 1, 285, 2119, 5, 8, 1, 5810, 3770, 7, 14661, 2, 1120, 73, 2, 6124, 952, 14643, 9, 110, 38, 6457, 270, 3], [1, 1, 193, 3585, 5, 2, 160, 6, 1152, 123, 4, 2563, 38, 1, 337, 1838, 735, 27, 176, 960, 2, 563, 3], [2, 149, 1835, 307, 20, 722, 6, 38, 4974, 158, 2258, 972, 16, 2, 44, 123, 3], [670, 623, 2, 4974, 140, 731, 50, 139, 390, 4, 1, 57, 6, 2, 2258, 32, 15030, 3], [50, 399, 27, 5810, 9631, 6, 2, 66, 148, 299, 484, 227, 7, 769, 27, 410, 9, 8706, 121, 85, 132, 27, 55, 779, 239, 79, 1641, 3], [1, 1, 605, 2, 899, 6, 2, 1, 340, 4, 1802, 7, 545, 8, 1, 1, 2106, 1463, 2, 4658, 953, 59, 2903, 

In [ ]:
train_padded_sequences = pad_sequences(train_sequences,padding='post',maxlen=128)
validation_padded_sequences = pad_sequences(validation_sequences,padding='post',maxlen=128)
test_padded_sequences = pad_sequences(test_sequences,padding='post',maxlen=128)

In [ ]:
token2index = tokenizer.word_index
index2token = {index:token for token,index in token2index.items()}

Tags --
* B- indicates the beginning of an entity.
* I- indicates a token inside an entity.
* O indicates a token outside of any named entity

In [ ]:
tag_names=dataset["train"].features["ner_tags"].feature.names
tag2index={tag:index for index,tag in enumerate(tag_names)}
index2tag={index:tag for tag,index in tag2index.items()}
print(tag_names)
print(index2tag)
print(tag2index)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}


In [ ]:
"""
When input sequences are padded to a fixed length (e.g., 128 tokens), the label sequences must also be padded to match this length.
This ensures that each token has a corresponding label, maintaining the one-to-one mapping required for training.
"""
training_padded_tags = pad_sequences(training_tags,padding='post',maxlen=128)
validation_padded_tags = pad_sequences(validation_tags,padding='post',maxlen=128)
test_padded_tags = pad_sequences(test_tags,padding='post',maxlen=128)
print(training_padded_tags.shape)

(14041, 128)


In [ ]:
"""
to_categorical() -- converts integer class vectors into binary class matrices (one-hot encoding).
For each integer in the input, it creates a binary vector of length equal to num_classes,
where the index corresponding to the integer is set to 1, and all other indices are set to 0.
9 -> number of possible NER tags (classes).
"""
training_categorical = tf.keras.utils.to_categorical(training_padded_tags,num_classes=len(tag2index))
validation_categorical = tf.keras.utils.to_categorical(validation_padded_tags,num_classes=len(tag2index))
test_categorical = tf.keras.utils.to_categorical(test_padded_tags,num_classes=len(tag2index))
print(training_categorical.shape)

(14041, 128, 9)


# 1. Bi-LSTMs

In [ ]:
model=Sequential(
    [
        layers.Embedding(len(token2index)+1,500),  # we add +1 because index 0 is often reserved for padding (empty tokens) , OUTPUT - (batch_size, sequence_length, 500)
        layers.Bidirectional(LSTM(128,activation='tanh',return_sequences=True)),
        layers.Bidirectional(LSTM(128,activation='tanh',return_sequences=True)),
        layers.TimeDistributed(layers.Dense(len(tag2index),activation='softmax')) #
    ]
)
"""
Bidirectional --
Learns patterns from both directions (past and future) in the sequence.
  return_sequences -- False (default)	--> Only last time step's output	--> Sequence classification (e.g., sentiment)
  return_sequences -- True	--> All time steps' outputs	--> Sequence labeling (e.g., NER)
"""

"\nBidirectional -- \nLearns patterns from both directions (past and future) in the sequence.\n  return_sequences -- False (default)\t--> Only last time step's output\t--> Sequence classification (e.g., sentiment) \n  return_sequences -- True\t--> All time steps' outputs\t--> Sequence labeling (e.g., NER)\n"

In [ ]:
model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=['accuracy'])

In [ ]:
model.fit(
    train_padded_sequences, training_categorical,
    batch_size=32,
    epochs=10,
    validation_data=(validation_padded_sequences, validation_categorical)
)

Epoch 1/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 757s 2s/step - accuracy: 0.9661 - loss: 0.1824 - val_accuracy: 0.9818 - val_loss: 0.0613
Epoch 2/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 747s 2s/step - accuracy: 0.9873 - loss: 0.0417 - val_accuracy: 0.9911 - val_loss: 0.0311
Epoch 3/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 802s 2s/step - accuracy: 0.9953 - loss: 0.0164 - val_accuracy: 0.9934 - val_loss: 0.0252
Epoch 4/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 797s 2s/step - accuracy: 0.9979 - loss: 0.0081 - val_accuracy: 0.9940 - val_loss: 0.0224
Epoch 5/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 790s 2s/step - accuracy: 0.9988 - loss: 0.0046 - val_accuracy: 0.9941 - val_loss: 0.0241
Epoch 6/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 733s 2s/step - accuracy: 0.9993 - loss: 0.0028 - val_accuracy: 0.9938 - val_loss: 0.0285
Epoch 7/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 733s 2s/step - accuracy: 0.9995 - loss: 0.0020 - val_accuracy: 0.9940 - val_loss: 0.0279
Epoch 8/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 736s 2s/step - accuracy: 0.9996 - loss: 0.0015 - val_accu

# 1. model.evaluate() -> Quick Keras Evaluation
* What it does : Computes loss and accuracy token-by-token.
* Good for : Quick check if the model is learning something.
* Limitation : It does not understand full entities (e.g., "Barack Obama" as one entity).
* Measures : Basic token-wise correctness.

# 2. classification_report(true_tags, pred_tags) -> Deep, Proper NER Evaluation
* What it does : Computes Precision, Recall, F1-Score based on entities, not just single tokens.
* Good for : Real-world evaluation, especially for NER, POS, chunking, etc.
* Understands : BIO tagging, full entity spans, entity boundaries.
* Measures : How well entire named entities are detected.

In [ ]:
predicted_tags_probs=model.predict(test_padded_sequences) # The model outputs predicted probability distributions over NER tags for each token, (num_test_sentences, 128, 9)
print(predicted_tags_probs[0])
predicted_tags=tf.argmax(predicted_tags_probs,axis=-1).numpy()
"""
tf.argmax picks the index of the highest probability at each token.
axis=-1 means "take the maximum across the last dimension" (the 9 classes).
So instead of a probability vector, you get a single integer tag ID per token.
output -- (num_test_sentences, 128)
"""
print(predicted_tags.shape)

108/108 ━━━━━━━━━━━━━━━━━━━━ 37s 341ms/step
[[9.9998027e-01 3.3319068e-06 6.8756570e-08 ... 2.7759256e-07
  1.1727116e-05 2.2172014e-06]
 [9.9999934e-01 3.2443836e-07 1.3005477e-09 ... 2.1519320e-08
  1.7863410e-07 1.0946332e-07]
 [9.5960888e-05 3.4001432e-04 1.0718736e-06 ... 8.1574959e-05
  3.6256970e-04 5.2318592e-06]
 ...
 [9.9999994e-01 6.0482502e-10 5.3845730e-12 ... 5.9368238e-10
  1.5757729e-08 8.4154141e-09]
 [9.9999982e-01 4.5324122e-09 5.1846596e-11 ... 4.9358939e-09
  9.2937050e-08 4.7829126e-08]
 [9.9999267e-01 2.2787695e-07 6.8741910e-09 ... 3.6690369e-07
  4.3223472e-06 1.5095895e-06]]
(3453, 128)


In [ ]:
print(predicted_tags[0].shape)

(128,)


In [ ]:
true_tags = [] # shape -- (number of sentences, variable number of tokens)

for i, seq in enumerate(test_padded_tags):  # Loop over all sentences (here just one)
    sentence_true_tags = []  # New list for this sentence

    for idx in seq[:len(test_tokens[i])]:  # Only loop over real tokens (not padding)
        tag_name = index2tag[idx]  # Convert index to tag name
        sentence_true_tags.append(tag_name)  # Add tag name to this sentence list

    true_tags.append(sentence_true_tags)  # Add this sentence list to final true_tags

In [ ]:
pred_tags = []  # to store all the predicted tag names

for i, seq in enumerate(predicted_tags):
    sentence_pred_tags = []  # tags for one sentence

    for idx in seq[:len(test_tokens[i])]:  # only consider real tokens, not padding
        tag_name = index2tag[idx]  # convert index to tag name
        sentence_pred_tags.append(tag_name)  # add tag name to current sentence

    pred_tags.append(sentence_pred_tags)  # add full sentence tags to final list

In [ ]:
from seqeval.metrics import classification_report
print(classification_report(true_tags,pred_tags))

              precision    recall  f1-score   support

         LOC       0.71      0.68      0.70      1668
        MISC       0.64      0.63      0.63       702
         ORG       0.67      0.56      0.61      1661
         PER       0.73      0.54      0.62      1617

   micro avg       0.69      0.60      0.64      5648
   macro avg       0.69      0.60      0.64      5648
weighted avg       0.70      0.60      0.64      5648



In [ ]:
model.evaluate(test_padded_sequences,test_categorical)

108/108 ━━━━━━━━━━━━━━━━━━━━ 37s 339ms/step - accuracy: 0.9915 - loss: 0.0461


[0.043581996113061905, 0.9921784400939941]

# 2. SIMPLE RNNs

In [ ]:
model1=Sequential(
    [
        layers.Embedding(len(token2index)+1,500),
        layers.Bidirectional(SimpleRNN(128,activation='tanh',return_sequences=True)),
        layers.Bidirectional(SimpleRNN(128,activation='tanh',return_sequences=True)),
        layers.TimeDistributed(layers.Dense(len(tag2index),activation='softmax'))
    ]
)

In [ ]:
model1.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=False),
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=['accuracy'])

In [ ]:
model1.fit(
    train_padded_sequences, training_categorical,
    batch_size=32,
    epochs=5,
    validation_data=(validation_padded_sequences, validation_categorical)
)

Epoch 1/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 341s 741ms/step - accuracy: 0.9674 - loss: 0.1273 - val_accuracy: 0.9895 - val_loss: 0.0377
Epoch 2/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 362s 697ms/step - accuracy: 0.9946 - loss: 0.0208 - val_accuracy: 0.9938 - val_loss: 0.0235
Epoch 3/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 321s 694ms/step - accuracy: 0.9991 - loss: 0.0056 - val_accuracy: 0.9942 - val_loss: 0.0235
Epoch 4/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 336s 727ms/step - accuracy: 0.9998 - loss: 0.0018 - val_accuracy: 0.9943 - val_loss: 0.0245
Epoch 5/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 321s 725ms/step - accuracy: 0.9999 - loss: 7.6843e-04 - val_accuracy: 0.9940 - val_loss: 0.0275


# 3. Bi-DIRECTIONAL GRU

In [ ]:
model2=Sequential(
    [
        layers.Embedding(len(token2index)+1,300),
        layers.Bidirectional(GRU(128,activation='tanh',return_sequences=True)),
        layers.Bidirectional(GRU(128,activation='tanh',return_sequences=True)),
        layers.TimeDistributed(layers.Dense(len(tag2index),activation='softmax'))
    ]
)

In [ ]:
model2.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=['accuracy'])

In [ ]:
model2.fit(
    train_padded_sequences, training_categorical,
    batch_size=32,
    epochs=5,
    validation_data=(validation_padded_sequences, validation_categorical)
)

Epoch 1/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 672s 1s/step - accuracy: 0.9658 - loss: 0.1984 - val_accuracy: 0.9876 - val_loss: 0.0439
Epoch 2/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 676s 1s/step - accuracy: 0.9912 - loss: 0.0292 - val_accuracy: 0.9926 - val_loss: 0.0266
Epoch 3/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 677s 1s/step - accuracy: 0.9967 - loss: 0.0124 - val_accuracy: 0.9942 - val_loss: 0.0213
Epoch 4/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 667s 1s/step - accuracy: 0.9982 - loss: 0.0065 - val_accuracy: 0.9942 - val_loss: 0.0219
Epoch 5/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 666s 1s/step - accuracy: 0.9989 - loss: 0.0041 - val_accuracy: 0.9944 - val_loss: 0.0241


In [ ]:
model2.evaluate(test_padded_sequences,test_categorical)

108/108 ━━━━━━━━━━━━━━━━━━━━ 29s 268ms/step - accuracy: 0.9922 - loss: 0.0308


[0.02979051135480404, 0.9927123785018921]

---
# Making a function for all this


In [ ]:
def make_prediction(model, index2tag, token2index, test_sentence):
    # Remove padding (0s)
    real_token_indexes = test_sentence[test_sentence > 0]

    # Predict
    probs = model(test_sentence.reshape(1, -1))
    predicted_tag_indexes = tf.argmax(probs[0], axis=-1).numpy()

    # Trim prediction to the actual token length
    predicted_tag_indexes = predicted_tag_indexes[:len(real_token_indexes)]

    # Map indexes to tokens and tags
    tokens = [index2token.get(idx, '<UNK>') for idx in real_token_indexes]
    predicted_tags = [index2tag[idx] for idx in predicted_tag_indexes]

    return tokens, predicted_tags

In [ ]:
make_prediction(model, index2tag, index2token, test_padded_sequences[0])

(['soccer',
  '-',
  'japan',
  'get',
  'lucky',
  'win',
  ',',
  'china',
  'in',
  'surprise',
  'defeat',
  '.'],
 ['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O'])